### 1.lzd_sea_mc.json

In [ ]:
  "epnet": [
    {"column_name": "lower_k"},
    {"column_name": "traffic_src"},
    {"column_name": "page_idx"}
  ],

### 2. add gatenu.py

In [ ]:
import tensorflow as tf
from tensorflow.contrib import layers
from model_layers import Activation, bn_layer
from run_config import RunConfig as cfg
class GateNU:
    def __init__(self,
                 hidden_unit,
                 is_rank_trainning,
                 pv_labels,
                 gamma=2.,
                 l2_reg=cfg.dnn_l2_reg,
                 scope='gate_nu'):
        self.hidden_unit = hidden_unit
        self._is_rank_training = is_rank_trainning
        self._pv_labels = pv_labels
        self.gamma = gamma
        self.l2_reg = l2_reg
        self.scope = scope
    def __call__(self, inputs):
        with tf.variable_scope(self.scope):
            output = layers.fully_connected(
                inputs,
                self.hidden_unit,
                activation_fn=Activation("swish"),
                normalizer_fn=layers.layer_norm,
                normalizer_params={'scale': True, 'center': True}
            )
            output = layers.fully_connected(
                output,
                self.hidden_unit,
                activation_fn=Activation("sigmoid"),
                normalizer_fn=layers.layer_norm,
                normalizer_params={'scale': True, 'center': True}
            )
            return self.gamma * output

### 3. model

In [ ]:
from gatenu import GateNU

self.gate_nu_summary = OrderedDict()


            net_size = int(self._net.shape[-1])
            self.domain = self.mark_tag(input_tensors.get("epnet"),"gate_nu_domain_layer")
            gate_nu = GateNU(net_size, self._is_rank_training, self._pv_labels, scope='domain_gate_nu')(tf.concat([self.domain, tf.stop_gradient(self._net)], axis=-1))
            self.gate_nu_summary['base_features_GateNU'] = gate_nu
            self._net = self._net * gate_nu
            self._net = tf.concat([self._urb_item_rele_cosine_fea,      # (?, 32)
                                   self._coins_search_feats,            # (?, 6)
                                   self._cross_feature_layer,           # (?, 64)
                                   self._net], axis=1)

### 4. summary

self.gate_nu_summary = OrderedDict()
self.gate_nu_summary['Hidden_{}_{}_GateNU'.format(layer_id, name)] = gate_nu

for name, var in self.gate_nu_summary.items():
    variable_summaries(model, name, var, mode)